<a href="https://colab.research.google.com/github/statisticianinstilettos/Hackathon-Assets/blob/main/Custom%20Evals%20Example/Monitoring_LLM_Systems_in_Production_with_Comet_Opik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LLM Observability and Monitoring Tutorial with my Recipe Generator App

This example personal project is designed to practice establishing proper montioring best practices for a live LLM app. In this tutorial, I'll walk you through how to build a very simple yet powerful LLM app, and show how to design the monitoring metrics and dashboards to facilitate detecting and root causing issues that might arise in production.


## Preparing our environment

First, we will install the required libraries and set up our OpenAI API keys. Check out Opik config parameters here: https://www.comet.com/docs/opik/tracing/sdk_configuration

In [ ]:
%pip install -U opik openai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.2/150.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.0/589.0 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.8/735.8 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.2 MB/s eta 0:00:00


In [ ]:
import opik

opik.configure(use_local=False)

OPIK: Your Opik API key is available in your account settings, can be found at https://www.comet.com/api/my/settings/ for Opik cloud


Please enter your Opik API key:··········
Do you want to use "statisticianinstilettos" workspace? (Y/n)Y


OPIK: Configuration saved to file: /root/.opik.config


In [ ]:
import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


In [ ]:
from opik import track, opik_context, Attachment
from opik.integrations.openai import track_openai
from openai import OpenAI

os.environ["OPIK_PROJECT_NAME"] = "recipe-generator"

client = OpenAI()
openai_client = track_openai(client)

# Building a simple recipe generator

I'm using the OpenAI API to generate a response, and Opik to track and evaluate it.

In [ ]:
#This function performs the API call to openAI to request the recipe based on the prompt. Simply enter the dish name!
#Because I'm using a function, I'm using the Opik @track decorator to log my traces.

@opik.track()
def generate_recipe(dish_name):

    prompt = f"""
    You are a world-class home chef and recipe developer. Generate a step-by-step recipe for {dish_name}.
    Include a list of ingredients with quantities, clear cooking instructions, preparation time, cooking time, and serving size.
    If applicable, provide useful tips, optional variations, or suggestions for substitutes.
    Keep the recipe straightforward and easy to follow for a novice at home cook.
    """

    # Request recipe generation from OpenAI's completions model
    response = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a world-class chef and recipe developer."},  # system message
            {"role": "user", "content": prompt}  # User's input prompt
        ],
        max_tokens=500,
        temperature=0.7,
        n=1
    )


    return response



In [ ]:
# Example usage
dish_name = "blt sandwitch"
recipe = generate_recipe(dish_name)


#extract just the recipe text and print
print(recipe.choices[0].to_dict()['message']['content'])

**BLT Sandwich Recipe**

**Ingredients:**
- 8 slices of bacon
- 2 large tomatoes, thinly sliced
- 4 leaves of lettuce
- 8 slices of sandwich bread
- Mayonnaise
- Salt and pepper to taste

**Preparation Time:** 10 minutes  
**Cooking Time:** 10 minutes  
**Serving Size:** 4 sandwiches  

**Instructions:**

**1. Cook the Bacon:**
   - In a large skillet over medium heat, cook the bacon until it is crispy, turning occasionally to ensure even cooking.
   - Once the bacon is cooked, transfer it to a plate lined with paper towels to drain excess grease.

**2. Toast the Bread:**
   - Toast the slices of sandwich bread until they are golden brown and crispy. You can use a toaster or a skillet for this step.

**3. Assemble the BLT Sandwich:**
   - Spread mayonnaise on one side of each slice of bread.
   - Place a leaf of lettuce on four slices of bread.
   - Top the lettuce with slices of tomato.
   - Season the tomatoes with a pinch of salt and pepper.
   - Add 2 slices of bacon on top of the 